In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import stratigraph as sg
from mayavi import mlab
import matplotlib.pyplot as plt
import numpy as np
import h5py
from tqdm import trange
import scipy.io as sio
from PIL import ImageFont
from PIL import ImageDraw
import os
import pandas as pd
import matplotlib as mpl
from matplotlib.colors import ListedColormap
import cmocean
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, InsetPosition
import matplotlib as mpl
from matplotlib import gridspec
from importlib import reload

# set up graphics:
%matplotlib qt
plt.rcParams['svg.fonttype'] = 'none'

### Load data

The data used below was generated at the St. Anthony Fals Laboratory, University of Minnesota, and is available from [this Zenodo repository](https://zenodo.org/records/10583965).

In [4]:
dirname = './data/XES_02_topography/'
T = np.zeros((261,111,101))
filenames = os.listdir(dirname)
for filename in filenames:
    surf_no = int(filename[8:-4])
    xyz = np.loadtxt(dirname+filename)
    z = xyz[:,2]
    T[:,:,surf_no-1] = np.reshape(z,(261,111))
dirname = './data/XES_02_basement_topography/'
B = np.zeros((261,111,101))
filenames = os.listdir(dirname)
for filename in filenames:
    surf_no = int(filename[:-4])
    xyz = np.loadtxt(dirname+filename)
    z = xyz[:,2]
    B[:,:,surf_no-1] = np.reshape(z,(261,111))

In [5]:
df = pd.read_csv('./data/XES_02_sealevel_and_scantimes.csv')
df[:10]

,Scan number,sl(mm),run time (hhh:mm:ss)
0,1,-129.9,0: 0: 0
1,2,-130.1,2: 6: 0
2,3,-130.5,2: 7: 0
3,4,-130.0,10: 0: 0
4,5,-130.2,18: 1: 8
5,7,-135.6,34: 0: 38
6,8,-151.7,41: 59: 59
7,9,-171.8,49: 10: 44
8,10,-200.3,58: 0: 9
9,11,-222.4,66: 0: 26


### Preprocess data

In [6]:
def convert_to_seconds(string):
    h = int(string.split(':')[0])
    m = int(string.split(':')[1])
    s = int(string.split(':')[2])
    return h*60*60 + m*60 + s

exp_time = np.nan * np.ones((T.shape[2],))
sea_level = np.nan * np.ones((T.shape[2],))

for i in range(len(exp_time)):
    if len(df.loc[df['Scan number'] == i+1]) > 0:
        exp_time[i] = convert_to_seconds(df.loc[df['Scan number'] == i+1]['run time (hhh:mm:ss)'].values[0])
        sea_level[i] = df.loc[df['Scan number'] == i+1]['sl(mm)']
        
        
missing_times = np.where(np.isnan(exp_time)==1)[0]
for i in missing_times:
    exp_time[i] = (exp_time[i-1] + exp_time[i+1]) * 0.5
    sea_level[i] = (sea_level[i-1] + sea_level[i+1]) * 0.5

exp_time = np.delete(exp_time, np.array([2, 5, 12, 25, 30, 31, 48, 60])) # get rid of locations with missing data
sea_level = np.delete(sea_level, np.array([2, 5, 12, 25, 30, 31, 48, 60])) # get rid of locations with missing data
    

plt.figure(figsize=(10, 6))
plt.plot(exp_time, sea_level, 'o-')
plt.xlabel('time (seconds)', fontsize = 16)
plt.ylabel('sea level (mm)', fontsize = 16);

T = np.delete(T, np.array([2, 5, 12, 25, 30, 31, 48, 60]), axis=2) # get rid of locations with no data
B = np.delete(B, np.array([2, 5, 12, 25, 30, 31, 48, 60]), axis=2) # get rid of locations with no data

/var/folders/wn/31h1q45j6y321jzk91br_d3m0000gn/T/ipykernel_96830/4116070009.py:13: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  sea_level[i] = df.loc[df['Scan number'] == i+1]['sl(mm)']


In [7]:
# create resampled and smooth sea level curve
# reload(sg)
sampling_rate = 3600 # resample at every 3600 seconds; this is better than higher frequency resampling
time1, sea_level_rs1 = sg.resample_elevation_spl(exp_time, sea_level, sampling_rate)
plt.figure()
plt.plot(exp_time, sea_level, '.-')
plt.plot(time1, sea_level_rs1, '.-')

time2, sea_level_rs2 = sg.resample_elevation_int1d(exp_time, sea_level, sampling_rate)
plt.plot(time2, sea_level_rs2, '.-')

sea_level_rs1[134:145] = sea_level_rs2[134:145] # replace sea level 1 w/ sea level 2
sea_level_rs1[162:203] = sea_level_rs2[162:203] # replace sea level 1 w/ sea level 2

plt.plot(time1, sea_level_rs1, 'b.-')

time = time1.copy()
sea_level_rs = sea_level_rs1.copy()

In [8]:
# resample topography and subsidence arrays:
topo = np.zeros((T.shape[0], T.shape[1], len(time)))
for j in range(T.shape[1]):
    for i in range(T.shape[0]):
        elevation = T[i, j, :].copy()
        time, elevation = sg.resample_elevation_int1d(exp_time, elevation, sampling_rate)
        topo[i,j,:] = elevation  
subsid = np.zeros((B.shape[0], B.shape[1], len(time)))
for j in range(B.shape[1]):
    for i in range(B.shape[0]):
        elevation = B[i, j, :].copy()
        time, elevation = sg.resample_elevation_int1d(exp_time, elevation, sampling_rate)
        subsid[i,j,:] = elevation

# adjust topographic and subsidence surfaces in the proximal corners (for aestethic reasons only)
inds = np.indices(np.shape(subsid[:,:,0]))
# inds2 = np.argwhere(inds[0] < -4.8*inds[1] + 120)
# inds3 = np.argwhere(inds[0] > 5.13*inds[1] + 142)
inds2 = np.argwhere(inds[0] < -5.3*inds[1] + 125)
inds3 = np.argwhere(inds[0] > 5.3*inds[1] + 125)
for i in range(np.shape(subsid)[2]):
    subsid[:, :, i][inds2[:,0], inds2[:,1]] = -229.6
    subsid[:, :, i][inds3[:,0], inds3[:,1]] = -229.6
for i in range(np.shape(topo)[2]):
    topo[:, :, i][inds2[:,0], inds2[:,1]] = 0
    topo[:, :, i][inds3[:,0], inds3[:,1]] = 0
    
# account for subsidence:
topo_s = topo.copy() 
for i in range(0, topo.shape[2]):
    topo_s[:,:,i] = topo_s[:,:,i]+(subsid[:,:,-1]-subsid[:,:,i])

In [12]:
np.shape(subsid)

(261, 111, 311)

In [11]:
plt.figure()
plt.plot(topo_s[120,:,0])
plt.plot(topo_s[120,:,150])
plt.plot(topo_s[120,:,-1]);

### Create Wheeler diagram(s)

In [9]:
strat, wheeler, wheeler_strat, vacuity = sg.create_wheeler_diagram(topo_s, 0.5)

### Figure 19: Time-elevation plots

In [10]:
# create a number of time-elevation plots, from proximal to distal locations
elevation = topo_s[125, 20, :].copy()
fig, dve_data, duration_thickness_data, new_ts_labels, strat_tops, strat_top_inds, bound_inds, interval_labels = \
        sg.plot_strat_diagram(elevation, 'mm', time, 'seconds', 0.5, 
        np.max(elevation), np.max(time), plotting=True)
elevation = topo_s[125, 40, :].copy()
fig, dve_data, duration_thickness_data, new_ts_labels, strat_tops, strat_top_inds, bound_inds, interval_labels = \
        sg.plot_strat_diagram(elevation, 'mm', time, 'seconds', 0.5, 
        np.max(elevation), np.max(time), plotting=True)
elevation = topo_s[125, 60, :].copy()
fig, dve_data, duration_thickness_data, new_ts_labels, strat_tops, strat_top_inds, bound_inds, interval_labels = \
        sg.plot_strat_diagram(elevation, 'mm', time, 'seconds', 0.5, 
        np.max(elevation), np.max(time), plotting=True)
elevation = topo_s[125, 80, :].copy()
fig, dve_data, duration_thickness_data, new_ts_labels, strat_tops, strat_top_inds, bound_inds, interval_labels = \
        sg.plot_strat_diagram(elevation, 'mm', time, 'seconds', 0.5, 
        np.max(elevation), np.max(time), plotting=True)

### Figure 20: Erosional characteristics of stratigraphic surfaces

In [11]:
erosional_surfs_age_below, erosional_surfs_age_above, erosional_surfs_time, erosional_surfs_thickness, erosional_surfs_age_of_first_erosion = \
        sg.compute_erosional_surf_attributes(strat, time, topo_s, erosion_threshold = 0.0001)

100%|██████████| 261/261 [00:19<00:00, 13.61it/s]


In [16]:
cmap = cmocean.cm.deep_r
newcmap = cmocean.tools.crop_by_percent(cmap, 30, which='min', N=None)

def plot_strat_surface_attributes(erosional_surfs_first_age, erosional_surfs_last_age, 
                                  erosional_surfs_time, erosional_surfs_thickness, ts, cmap):
    fig = plt.figure(figsize = (18, 10))
    gs = gridspec.GridSpec(2, 2, left=0.05, right=0.98, wspace=0.001)
    
    ax1 = fig.add_subplot(gs[0, 0])
    temp = erosional_surfs_first_age[:, :, ts].copy()
    temp[temp==-1] = np.nan
    im = ax1.imshow(temp, aspect=0.2, interpolation='none', cmap=cmap, 
                          vmin=0, vmax=310)
    ax1.contour(strat[:,:,ts], colors='k', levels=200, linestyles='solid', linewidths=0.3)
    ax1.set_xticks([])
    ax1.set_yticks([])
    ax1.set_title('age of deposits below')
    cbar = fig.colorbar(im, shrink=1, ax=ax1)
    cbar.ax.get_yaxis().labelpad = 15
    cbar.ax.set_ylabel('age (hours)', rotation=270)
    
    ax2 = fig.add_subplot(gs[0, 1])
    temp = erosional_surfs_last_age[:, :, ts].copy()
    temp[temp==-1] = np.nan
    im = ax2.imshow(temp, aspect=0.2, interpolation='none', cmap=cmap, 
                          vmin=0, vmax=310)
    ax2.contour(strat[:,:,ts], colors='k', levels=200, linestyles='solid', linewidths=0.3)
    ax2.set_xticks([])
    ax2.set_yticks([])
    ax2.set_title('age of deposits above')
    cbar = fig.colorbar(im, shrink=1, ax=ax2)
    cbar.ax.get_yaxis().labelpad = 15
    cbar.ax.set_ylabel('age (hours)', rotation=270)
    
    ax3 = fig.add_subplot(gs[1, 0])
    temp = erosional_surfs_time[:, :, ts].copy()
    temp[temp==-1] = np.nan
    im = ax3.imshow(temp, aspect=0.2, interpolation='none', cmap=cmap, 
                          vmin=0, vmax=310)
    ax3.contour(strat[:,:,ts], colors='k', levels=200, linestyles='solid', linewidths=0.3)
    ax3.set_xticks([])
    ax3.set_yticks([])
    ax3.set_title('time gap')
    cbar = fig.colorbar(im, shrink=1, ax=ax3)
    cbar.ax.get_yaxis().labelpad = 15
    cbar.ax.set_ylabel('time gap (hours)', rotation=270)
    
    ax4 = fig.add_subplot(gs[1, 1])
    temp = erosional_surfs_thickness[:, :, ts].copy()
    temp[temp==-1] = np.nan
    im = ax4.imshow(temp, aspect=0.2, interpolation='none', cmap=cmap, 
                          vmin=np.nanmin(temp), vmax=np.nanmax(temp))
    ax4.contour(strat[:,:,ts], colors='k', levels=200, linestyles='solid', linewidths=0.3)
    ax4.set_xticks([])
    ax4.set_yticks([])
    ax4.set_title('thickness eroded')
    cbar = fig.colorbar(im, shrink=1, ax=ax4)
    cbar.ax.get_yaxis().labelpad = 15
    cbar.ax.set_ylabel('thickness (mm)', rotation=270)
    
    sl_ax = plt.axes([0,0,1,1])
    ip = InsetPosition(ax1, [0.0, -0.23, 0.25, 0.22])
    sl_ax.set_axes_locator(ip)
    sl_ax.plot(time, sea_level_rs, 'k', linewidth=2)
    sl_ax.plot(time[ts], sea_level_rs[ts], 'ro')
    sl_ax.set_xlim(0, time[-1])
    sl_ax.set_ylim(np.min(sea_level_rs)-20, np.max(sea_level_rs)+20)
    sl_ax.set_xticks([]) #[3600*50*tick for tick in [0, 1, 2, 3, 4, 5, 6]])
    sl_ax.set_xticklabels([]) #[label*50 for label in [0, 1, 2, 3, 4, 5, 6]])
    sl_ax.set_yticks([])
    
    sl_ax = plt.axes([0,0,1,1])
    ip = InsetPosition(ax2, [0.0, -0.23, 0.25, 0.22])
    sl_ax.set_axes_locator(ip)
    sl_ax.plot(time, sea_level_rs, 'k', linewidth=2)
    sl_ax.plot(time[ts], sea_level_rs[ts], 'ro')
    sl_ax.set_xlim(0, time[-1])
    sl_ax.set_ylim(np.min(sea_level_rs)-20, np.max(sea_level_rs)+20)
    sl_ax.set_xticks([]) #[3600*50*tick for tick in [0, 1, 2, 3, 4, 5, 6]])
    sl_ax.set_xticklabels([]) #[label*50 for label in [0, 1, 2, 3, 4, 5, 6]])
    sl_ax.set_yticks([])
    
    sl_ax = plt.axes([0,0,1,1])
    ip = InsetPosition(ax3, [0.0, -0.23, 0.25, 0.22])
    sl_ax.set_axes_locator(ip)
    sl_ax.plot(time, sea_level_rs, 'k', linewidth=2)
    sl_ax.plot(time[ts], sea_level_rs[ts], 'ro')
    sl_ax.set_xlim(0, time[-1])
    sl_ax.set_ylim(np.min(sea_level_rs)-20, np.max(sea_level_rs)+20)
    sl_ax.set_xticks([]) #[3600*50*tick for tick in [0, 1, 2, 3, 4, 5, 6]])
    sl_ax.set_xticklabels([]) #[label*50 for label in [0, 1, 2, 3, 4, 5, 6]])
    sl_ax.set_yticks([])
    
    sl_ax = plt.axes([0,0,1,1])
    ip = InsetPosition(ax4, [0.0, -0.23, 0.25, 0.22])
    sl_ax.set_axes_locator(ip)
    sl_ax.plot(time, sea_level_rs, 'k', linewidth=2)
    sl_ax.plot(time[ts], sea_level_rs[ts], 'ro')
    sl_ax.set_xlim(0, time[-1])
    sl_ax.set_ylim(np.min(sea_level_rs)-20, np.max(sea_level_rs)+20)
    sl_ax.set_xticks([]) #[3600*50*tick for tick in [0, 1, 2, 3, 4, 5, 6]])
    sl_ax.set_xticklabels([]) #[label*50 for label in [0, 1, 2, 3, 4, 5, 6]])
    sl_ax.set_yticks([])
    
    return fig

In [17]:
ts = 207 # change this index to get maps for different time steps
fig = plot_strat_surface_attributes(erosional_surfs_age_below, erosional_surfs_age_above, 
                                  erosional_surfs_time, erosional_surfs_thickness, ts, newcmap)

### Plot cross sections from the 3D Wheeler diagram

In [18]:
rdbu = mpl.colormaps['RdBu'].resampled(256)
newcolors = rdbu(np.linspace(0, 1, 256))
newcolors[126:131, :] = np.array([1, 1, 1, 1])
wheelercmap = ListedColormap(newcolors)

In [19]:
loc = 130
fig = plt.figure(figsize=(15,15))
spec = gridspec.GridSpec(ncols=2, nrows=1,
                         width_ratios=[1, 6], wspace=0.0,
                         hspace=0.5) 
ax0 = fig.add_subplot(spec[0])
ax1 = fig.add_subplot(spec[1], sharey = ax0)
im = ax1.imshow(wheeler[loc,:,:].T, extent = [0, 111, time[-1], 0], cmap=wheelercmap, vmin = -6, vmax = 6, 
           interpolation='none', aspect='auto')
ax1.invert_yaxis()
plt.setp(ax1.get_yticklabels(), visible=False)
plt.colorbar(im);
ax0.plot(sea_level_rs, time, 'k', linewidth=3)
ax0.invert_xaxis()
ax0.set_xticks([-100, -200, -300])
ax0.set_xlim(-100, -370)
ax1.set_ylim(0, time[-1])
ax1.set_xlim(2, 111)
ax1.set_title('chronostratigraphic diagram', fontsize=20);

In [20]:
loc = 30
fig = plt.figure(figsize=(15,15))
spec = gridspec.GridSpec(ncols=2, nrows=1,
                         width_ratios=[1, 6], wspace=0.0,
                         hspace=0.5) 
ax0 = fig.add_subplot(spec[0])
ax1 = fig.add_subplot(spec[1], sharey = ax0)
im = ax1.imshow(wheeler[:,loc,:].T, extent = [0, 261*10, time[-1], 0], cmap=wheelercmap, vmin = -6, vmax = 6, 
           interpolation='none', aspect='auto') 
ax1.invert_yaxis()
plt.setp(ax1.get_yticklabels(), visible=False)
plt.colorbar(im);
ax0.plot(sea_level_rs, time, 'k', linewidth=3)
ax0.invert_xaxis()
ax0.set_xticks([-100, -200, -300])
ax0.set_xlim(-100, -370)
ax1.set_ylim(0, time[-1])
ax1.set_xlim(0, 2580)
ax1.set_title('chronostratigraphic diagram', fontsize=20);

### Figure 16A: Single dip section with Wheeler diagram

In [12]:
rdbu = mpl.colormaps['RdBu'].resampled(256)
newcolors = rdbu(np.linspace(0, 1, 256))
newcolors[126:131, :] = np.array([1, 1, 1, 1])
newcmp = ListedColormap(newcolors)

end_time = 311
loc = 130
dx = 50
ve = 1
fig = plt.figure(figsize=(15,12))
spec = gridspec.GridSpec(ncols=3, nrows=2,
                         width_ratios=[1, 6, 0.2], wspace=0.04,
                         hspace=0.05, height_ratios=[3, 2])
ax0 = fig.add_subplot(spec[1,0])
ax1 = fig.add_subplot(spec[1,1], sharey = ax0)
ax2 = fig.add_subplot(spec[0,1], sharex = ax1)
ax3 = fig.add_subplot(spec[1,2])
im = ax1.imshow(wheeler_strat[loc, :, :].T, cmap=newcmp, vmin = -10, vmax = 10, extent = [0, dx*(strat.shape[1]-1), 
        time[-1]/3600, 0], interpolation='none', aspect='auto')
ax1.invert_yaxis()
ax1.set_xlabel('distance (mm)', fontsize = 14)
ax2.set_ylabel('depth (mm)', fontsize = 14)
ax0.set_ylabel('time (hours)', fontsize = 14)
ax0.set_xlabel('base level (mm)', fontsize = 14)
ax1.set_xlim(115, 5200)
ax2.set_ylim(-1273, -45)
plt.setp(ax1.get_yticklabels(), visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)
plt.colorbar(im, cax=ax3)
ax3.text(2.5, -8, 'erosion (mm)', rotation=90)
ax3.text(2.5, 1.8, 'deposition (mm)', rotation=90)
cmap = mpl.colormaps['viridis']
norm = mpl.colors.Normalize(vmin=0, vmax=len(time)-1)

for i in range(len(time)-1):
    ax0.plot([sea_level_rs[i], sea_level_rs[i+1]], [time[i]/3600, time[i+1]/3600], 
             color=cmap(norm(i))[:3], linewidth=3)

sg.plot_dip_section(topo[:,:,:end_time], sg.topostrat(topo[:,:,:end_time]), 
                    dx, loc, ve, ax=ax2, sea_level = sea_level_rs[:end_time], subsid=subsid, 
                    linewidth=0.1, line_freq=1, color_mode='bathymetry', water_depth=100, plot_type='2D', 
                plot_erosion=True, erosional_surfs_thickness=erosional_surfs_thickness, 
                    plot_water=True, plot_basement=True)

ax2.plot([40*dx, 40*dx], [-1270, -100], 'k')
ax2.set_ylim(-1270, -100);

###  Animation (through time) of a dip section (animation of Figure 16A)

In [24]:
# this takes a while to run

loc = 130
dx = 50
ve = 1

for end_time in range(3, 312, 3):
    # account for subsidence:
    topo_s_temp = topo[:,:,:end_time].copy() 
    for i in range(0, topo[:,:,:end_time].shape[2]):
        topo_s_temp[:,:,i] = topo_s_temp[:,:,i]+(subsid[:,:,:end_time][:,:,-1]-subsid[:,:,:end_time][:,:,i])
    
    strat_temp, wheeler_temp, wheeler_strat_temp, vacuity_temp = sg.create_wheeler_diagram(topo_s_temp, 0.5)

    erosional_surfs_age_below, erosional_surfs_age_above, erosional_surfs_time, erosional_surfs_thickness =\
        sg.compute_erosional_surf_attributes(strat_temp, time[:end_time], topo_s_temp)
    
    fig = plt.figure(figsize=(15,12))
    spec = gridspec.GridSpec(ncols=3, nrows=2,
                             width_ratios=[1, 6, 0.2], wspace=0.04,
                             hspace=0.05, height_ratios=[3, 2])
    ax0 = fig.add_subplot(spec[1,0])
    ax1 = fig.add_subplot(spec[1,1], sharey = ax0)
    ax2 = fig.add_subplot(spec[0,1], sharex = ax1)
    ax3 = fig.add_subplot(spec[1,2])
    im = ax1.imshow(wheeler_strat_temp[loc, :, :].T, cmap=newcmp, vmin = -10, vmax = 10, 
                    extent = [0, dx*(strat.shape[1]-1), 
                    time[:end_time][-1]/3600, 0], interpolation='none', aspect='auto')
    ax1.invert_yaxis()
    ax1.set_ylim(0, time[-1]/3600)
    ax1.set_xlabel('distance (mm)', fontsize = 14)
    ax2.set_ylabel('depth (mm)', fontsize = 14)
    ax0.set_ylabel('time (hours)', fontsize = 14)
    ax0.set_xlabel('sea level (mm)', fontsize = 14)
    ax0.set_xlim(-350, -120)
    ax1.set_xlim(115, 5200)
    ax2.set_ylim(-1273, 0)
    plt.setp(ax1.get_yticklabels(), visible=False)
    plt.setp(ax2.get_xticklabels(), visible=False)
    plt.colorbar(im, cax=ax3)
    ax3.text(2.5, -8, 'erosion (mm)', rotation=90)
    ax3.text(2.5, 1.8, 'deposition (mm)', rotation=90)
    cmap = mpl.colormaps['viridis']
    norm = mpl.colors.Normalize(vmin=0, vmax=len(time)-1)

    # plot sea level curve:
    for i in range(len(time[:end_time])-1):
        ax0.plot([sea_level_rs[i], sea_level_rs[i+1]], [time[i]/3600, time[i+1]/3600], 
                 color=cmap(norm(i))[:3], linewidth=3)

    sg.plot_dip_section(topo[:,:,:end_time], strat_temp, 
                        dx, loc, ve, ax=ax2, sea_level = sea_level_rs[:end_time], subsid=subsid[:,:,:end_time], 
                        linewidth=0.1, line_freq=1, color_mode='bathymetry', plot_type='2D', 
                        plot_erosion=True, erosional_surfs_thickness=erosional_surfs_thickness, 
                        plot_water=True, plot_basement=True)
    
    # add extra gray area at bottom of depth section:
    x = [115, 5200, 5200, 115]
    y = [np.nanmin(strat_temp[loc, :, :])-20, np.nanmin(strat_temp[loc, :, :])-20, -1273, -1273]
    ax2.fill(x, y, color='lightgray')
    
    fname = '/Users/zoltan/Dropbox/Chronostratigraphy/XES02_3D_dip_section_130_'+'%03d.png'%(end_time//3)
    fig.savefig(fname, dpi=300)
    plt.close('all')

100%|█████████████████████████████████████████| 261/261 [00:17<00:00, 14.95it/s]


### Figure 16B: Single strike section with Wheeler diagram

In [17]:
reload(sg)
end_time = 311
# account for subsidence:
topo_s_temp = topo[:,:,:end_time].copy() 
for i in range(0, topo[:,:,:end_time].shape[2]):
        topo_s_temp[:,:,i] = topo_s_temp[:,:,i]+(subsid[:,:,:end_time][:,:,-1]-subsid[:,:,:end_time][:,:,i])
strat_temp, wheeler_temp, wheeler_strat_temp, vacuity_temp = sg.create_wheeler_diagram(topo_s_temp, 0.5)

loc = 40
dx = 10
fig = plt.figure(figsize=(15,12))
spec = gridspec.GridSpec(ncols=3, nrows=2,
                         width_ratios=[1, 6, 0.2], wspace=0.04,
                         hspace=0.05, height_ratios=[3, 2])
ax0 = fig.add_subplot(spec[1,0])
ax1 = fig.add_subplot(spec[1,1], sharey = ax0)
ax2 = fig.add_subplot(spec[0,1], sharex = ax1)
ax3 = fig.add_subplot(spec[1,2])
im = ax1.imshow(wheeler_strat[:, loc, :].T, cmap=newcmp, vmin = -10, vmax = 10, extent = [0, dx*(strat.shape[0]-1), 
        time[-1]/3600, 0], interpolation='none', aspect='auto')
ax1.invert_yaxis()
ax1.set_xlabel('distance (mm)', fontsize = 14)
ax2.set_ylabel('depth (mm)', fontsize = 14)
ax0.set_ylabel('time (hours)', fontsize = 14)
ax0.set_xlabel('base level (mm)', fontsize = 14)
ax1.set_xlim(0, 2575)
ax2.set_ylim(-640, -200)
plt.setp(ax1.get_yticklabels(), visible=False)
plt.setp(ax2.get_xticklabels(), visible=False)
plt.colorbar(im, cax=ax3)
ax3.text(2.5, -8, 'erosion (mm)', rotation=90)
ax3.text(2.5, 1.8, 'deposition (mm)', rotation=90)
cmap = mpl.colormaps['viridis']
norm = mpl.colors.Normalize(vmin=0, vmax=len(time)-1)

for i in range(len(time)-1):
    ax0.plot([sea_level_rs[i], sea_level_rs[i+1]], [time[i]/3600, time[i+1]/3600], 
             color=cmap(norm(i))[:3], linewidth=3)

sg.plot_strike_section(topo[:,:,:end_time], strat_temp, 
                    dx, loc, ve, ax=ax2, sea_level = sea_level_rs[:end_time], subsid=subsid[:,:,:end_time], 
                    linewidth=0.1, line_freq=1, color_mode='age', plot_type='2D', 
                    plot_erosion=True, erosional_surfs_thickness=erosional_surfs_thickness, 
                    plot_water=True, plot_basement=True)

ax2.plot([130*dx, 130*dx], [-1300, -100], 'k');

###  Animation (through time) of a strike section (animation of Figure 16B)

In [ ]:
reload(sg)
loc = 40
dx = 10

for end_time in range(311, 312, 3):
    # account for subsidence:
    topo_s_temp = topo[:,:,:end_time].copy() 
    for i in range(0, topo[:,:,:end_time].shape[2]):
        topo_s_temp[:,:,i] = topo_s_temp[:,:,i]+(subsid[:,:,:end_time][:,:,-1]-subsid[:,:,:end_time][:,:,i])
    
    strat_temp, wheeler_temp, wheeler_strat_temp, vacuity_temp = sg.create_wheeler_diagram(topo_s_temp, 0.5)

    erosional_surfs_age_below, erosional_surfs_age_above, erosional_surfs_time, erosional_surfs_thickness =\
        sg.compute_erosional_surf_attributes(strat_temp, time[:end_time], topo_s_temp)

    fig = plt.figure(figsize=(15,12))
    spec = gridspec.GridSpec(ncols=3, nrows=2,
                             width_ratios=[1, 6, 0.2], wspace=0.04,
                             hspace=0.05, height_ratios=[3, 2])
    ax0 = fig.add_subplot(spec[1,0])
    ax1 = fig.add_subplot(spec[1,1], sharey = ax0)
    ax2 = fig.add_subplot(spec[0,1], sharex = ax1)
    ax3 = fig.add_subplot(spec[1,2])
    im = ax1.imshow(wheeler_strat_temp[:, loc, :].T, cmap=newcmp, vmin = -10, vmax = 10, 
            extent = [0, dx*(strat.shape[0]-1), 
            time[:end_time][-1]/3600, 0], interpolation='none', aspect='auto')
    ax1.invert_yaxis()
    ax0.set_xlim(-350, -120)
    ax1.set_xlabel('distance (mm)', fontsize = 14)
    ax2.set_ylabel('depth (mm)', fontsize = 14)
    ax0.set_ylabel('time (hours)', fontsize = 14)
    ax0.set_xlabel('sea level (mm)', fontsize = 14)
    ax1.set_xlim(0, 2575)
    ax1.set_ylim(0, time[-1]/3600)
    ax2.set_ylim(-640, 0)
    plt.setp(ax1.get_yticklabels(), visible=False)
    plt.setp(ax2.get_xticklabels(), visible=False)
    plt.colorbar(im, cax=ax3)
    ax3.text(2.5, -8, 'erosion (mm)', rotation=90)
    ax3.text(2.5, 1.8, 'deposition (mm)', rotation=90)
    cmap = mpl.colormaps['viridis']
    norm = mpl.colors.Normalize(vmin=0, vmax=len(time)-1)

    for i in range(len(time[:end_time])-1):
        ax0.plot([sea_level_rs[i], sea_level_rs[i+1]], [time[i]/3600, time[i+1]/3600], 
                 color=cmap(norm(i))[:3], linewidth=3)

    sg.plot_strike_section(topo[:,:,:end_time], strat_temp, 
                        dx, loc, ve, ax=ax2, sea_level = sea_level_rs[:end_time], subsid=subsid[:,:,:end_time], 
                        linewidth=0.1, line_freq=1, color_mode='bathymetry', plot_type='2D', 
                        plot_erosion=True, erosional_surfs_thickness=erosional_surfs_thickness, 
                        plot_water=True, plot_basement=True)
    
    # add extra gray area at bottom of depth section:
    miny = np.nanmin(strat_temp[loc, :, :])-20
    if miny > -640:
        x = [0, 2575, 2575, 0]
        y = [miny, miny, -640, -640]
        ax2.fill(x, y, color='lightgray')
    
    fname = '/Users/zoltan/Dropbox/Chronostratigraphy/XES02_3D_strike_section_40_'+'%03d.png'%(end_time//3)
    fig.savefig(fname, dpi=300)
    plt.close('all')

### Figure 17: Plan-view sections of the Wheeler diagram

In [13]:
def plot_time_step(strat, wheeler, topo, sea_level_rs, ts, cmap, fig, ax, shrink):
    im = ax.imshow(wheeler[:, :, ts-1], cmap=cmap, vmin = -60, vmax = 60, 
               extent = [0, 50*strat.shape[1], 0, 10*strat.shape[0]], interpolation='none')
    x, y = np.meshgrid(np.arange(0, 50*strat.shape[1], 50), np.arange(0, 10*strat.shape[0], 10))
    temp = sg.sgolay2d(topo[:, :, ts], 5, 3)
    ax.contour(x, y, temp[::-1,:], colors='k', linewidths=0.2, linestyles ='solid', levels=np.arange(-1100,0,5))
    ax.contour(x, y, temp[::-1,:], colors='k', linewidths=2, linestyles ='dashed', 
                levels=[sea_level_rs[ts]])
    ax.set_xlim(0, 5500)
    ax.set_ylim(2600, 0)
    fig.colorbar(im, ax=ax, shrink=shrink, pad = 0.02)

In [14]:
rdbu = mpl.colormaps['RdBu'].resampled(256)
newcolors = rdbu(np.linspace(0, 1, 256))
newcolors[126:131, :] = np.array([1, 1, 1, 1])
rdbucmp = ListedColormap(newcolors)

fig, axes = plt.subplots(3, 2, figsize=(15, 10))
count = 0
for ts in range(203, 203+3*6, 3):
    plot_time_step(strat, wheeler, topo, sea_level_rs, ts, rdbucmp, fig, axes[count//2, count%2], shrink=0.97)
    count += 1

### 3D visualization of Wheeler diagram with plane widgets

In [20]:
mlab.figure()
source = mlab.pipeline.scalar_field(np.swapaxes(wheeler, 0, 1))
source.spacing = [5, 1, 1]
for axis in ['x', 'y', 'z']:
    plane = mlab.pipeline.image_plane_widget(source, plane_orientation = '{}_axes'.format(axis),
                                           slice_index=i, colormap='RdBu', vmin = -30, vmax = 30);

### Figure 18: Map of stratigraphic attributes

In [17]:
# create 3D facies array (as a function of water depth in this case)
ny, nx, nz = np.shape(strat)
facies = np.zeros((ny, nx, nz))
for i in range(facies.shape[2]):
    topo_sl = topo[:, :, i] - sea_level_rs[i]
    facies_sl = np.zeros(np.shape(topo_sl))
    facies_sl[topo_sl >= 0] = 0
    facies_sl[(topo_sl < 0) & (topo_sl >= -100)] = 1
    facies_sl[topo_sl < -100] = 2
    facies[:, :, i] = facies_sl
    
# eliminating nans from 'topo' array
topo[125,0,243] = topo[126,0,243]
topo[125,0,244] = topo[126,0,244]
topo[125,0,245] = topo[126,0,245]

# eliminating nans from 'strat' array
for i in range(np.shape(strat)[2]):
    t = strat[:, :, i]
    t[np.isnan(t) == 1] = t[136, 1]
    strat[:, :, i] = t

In [18]:
# resample topo, facies, strat, and subsid arrays to 500 x 1010 pixels
from skimage.transform import resize
strat2 = np.zeros((500, 1000, strat.shape[2]))
facies2 = np.zeros((500, 1000, strat.shape[2]))
topo2 = np.zeros((500, 1000, topo.shape[2]))
subsid2 = np.zeros((500, 1000, topo.shape[2]))
for i in trange(strat.shape[2]):
    strat2[:, :, i] = resize(strat[:, :90, i], (500, 1010), mode='reflect')[:,5:-5]
    facies2[:, :, i] = resize(facies[:, :90, i], (500, 1010), mode='reflect')[:,5:-5]
    topo2[:, :, i] = resize(topo[:, :90, i], (500, 1010), mode='reflect')[:,5:-5]
    subsid2[:, :, i] = resize(subsid[:, :90, i], (500, 1010), mode='reflect')[:,5:-5]

100%|██████████| 311/311 [00:17<00:00, 18.03it/s]


In [19]:
topo_s = topo2.copy() 
for i in range(0, topo_s.shape[2]):
    topo_s[:,:,i] = topo_s[:,:,i]+(subsid2[:,:,-1]-subsid2[:,:,i])

In [20]:
strat, wheeler, wheeler_strat, vacuity = sg.create_wheeler_diagram(topo_s, 0.5)

In [25]:
deposition_time, erosion_time, stasis_time, vacuity_time, deposition_thickness, erosion_thickness =\
        sg.compute_strat_maps(strat2, wheeler, wheeler_strat, vacuity)

# set 'uninteresting' areas to NaN:
deposition_time[stasis_time == 1] = np.nan
erosion_time[stasis_time == 1] = np.nan
vacuity_time[stasis_time == 1] = np.nan
deposition_thickness[stasis_time == 1] = np.nan
erosion_thickness[stasis_time == 1] = np.nan
stasis_time[stasis_time == 1] = np.nan

import cmocean
fig, axs = plt.subplots(2, 3, sharey=True, figsize=(20, 8))
im = axs[0,0].imshow(deposition_time, vmin=0, vmax=1, cmap=cmocean.cm.deep_r)
axs[0,0].contour(deposition_time, levels=np.linspace(0,1,10), colors='k', linewidths=0.5)
axs[0,0].set_title('deposition (time)')
axs[0,0].set_xticks([])
axs[0,0].set_yticks([])
fig.colorbar(im, ax=axs[0,0], shrink=0.65)
im = axs[0,1].imshow(erosion_time, vmin=0, vmax=0.5, cmap=cmocean.cm.deep_r)
axs[0,1].contour(erosion_time, levels=np.linspace(0,0.5,5), colors='k', linewidths=0.5)
axs[0,1].set_title('erosion (time)')
axs[0,1].set_xticks([])
axs[0,1].set_yticks([])
fig.colorbar(im, ax=axs[0,1], shrink=0.65)
im = axs[0,2].imshow(stasis_time, vmin=0, vmax=0.5, cmap=cmocean.cm.deep_r)
axs[0,2].contour(stasis_time, levels=np.linspace(0,0.5,5), colors='k', linewidths=0.5)
axs[0,2].set_title('stasis (time)')
axs[0,2].set_xticks([])
axs[0,2].set_yticks([])
fig.colorbar(im, ax=axs[0,2], shrink=0.65)
im = axs[1,0].imshow(vacuity_time, vmin=0, vmax=1, cmap=cmocean.cm.deep_r)
axs[1,0].contour(vacuity_time, levels=np.linspace(0,1,10), colors='k', linewidths=0.5)
axs[1,0].set_title('vacuity (time)')
axs[1,0].set_xticks([])
axs[1,0].set_yticks([])
fig.colorbar(im, ax=axs[1,0], shrink=0.65)
im = axs[1,1].imshow(deposition_thickness, vmin=40, vmax=770, cmap=cmocean.cm.deep_r)
axs[1,1].contour(deposition_thickness, levels=np.linspace(40,770,20), colors='k', linewidths=0.5)
axs[1,1].set_title('deposition (thickness)')
axs[1,1].set_xticks([])
axs[1,1].set_yticks([])
fig.colorbar(im, ax=axs[1,1], shrink=0.65)
im = axs[1,2].imshow(-erosion_thickness, vmin=0, vmax=600, cmap=cmocean.cm.deep_r)
axs[1,2].contour(-erosion_thickness, levels=np.linspace(0,600,20), colors='k', linewidths=0.5)
axs[1,2].set_title('erosion (thickness)')
axs[1,2].set_xticks([])
axs[1,2].set_yticks([])
fig.colorbar(im, ax=axs[1,2], shrink=0.65)
fig.tight_layout()

### Figure 15: Create 3D plot with arbitrarily oriented sections

The photos used here are available from: https://www.dropbox.com/scl/fo/b55cmom124n72ipw5xe9a/h?rlkey=vwxxhsvowm00xcpmc5ytvg0fd&dl=0

In [ ]:
# this takes a while to run (especially if you run the loop over all the time steps)
import glob
photo_times = []
fnames = glob.glob('/Users/zoltan/Documents/Chronostratigraphy/XES_02/R02_1 Overhead Photos/\
time lapse corrected photos/all_photos/*.tif')
fnames.sort()
for fname in fnames:
    photo_times.append(int(fname.split('/')[-1][8:15]))
photo_times = np.array(photo_times)

mlab.figure(bgcolor = (1,1,1))

from matplotlib import colors

for end_time in range(310, 311): #len(time)):
    mlab.clf()
    ind = np.argmin(np.abs(photo_times-time[end_time-1]))
    fname = fnames[ind]
    im = plt.imread(fname)
    model, kmeans_colors = sg.pick_colors(im, 256)
    im_pred = model.predict(im.reshape(-1, 3))
    im_pred = im_pred.reshape(im.shape[0], im.shape[1])
    im_pred = im_pred[::-1,:]

    temp = np.zeros(np.shape(topo2[2:491,50:-50,end_time-1]))
    inds = np.indices(np.shape(topo2[2:491,50:-50,end_time-1]))
    x1 = 0
    y1 = 268
    x2 = 300
    y2 = 488
    slope, y_intercept = sg.line_coefficients(x1, y1, x2, y2)
    inds2 = np.argwhere(inds[0] > slope*inds[1] + y_intercept)
    x1 = 0
    y1 = 212
    x2 = 300
    y2 = 0
    slope, y_intercept = sg.line_coefficients(x1, y1, x2, y2)
    inds3 = np.argwhere(inds[0] < slope*inds[1] + y_intercept)
    temp[inds2[:,0], inds2[:,1]] = 1
    temp[inds3[:,0], inds3[:,1]] = 1

    ve = 3
    dx = 10.0
    bottom = -1300
    scale = 1
    color_list = [(0,0,0), (0,0,0)]

    # account for subsidence:
    topo_s = topo2[2:491,50:-50,:end_time].copy() 
    for i in range(0, topo_s.shape[2]):
        topo_s[:,:,i] = topo_s[:,:,i]+(subsid2[2:491,50:-50,:end_time][:,:,-1]-subsid2[2:491,50:-50,:end_time][:,:,i])

    strat_ts = sg.topostrat(topo_s)

    x1 = 0
    y1 = 268
    x2 = 300
    y2 = 488
    s1 = 0
    sg.plot_random_section_2_points(topo2[2:491,50:-50,:end_time], strat_ts, dx, x1, x2, y1, y2, s1, ve, 
        bottom = bottom, xoffset=0, yoffset=0, sea_level=sea_level_rs, subsid=subsid2[2:491,50:-50,:end_time], 
        linewidth=1, line_freq=2, color_mode='bathymetry', plot_type='3D', plot_basement=True)

    x1 = 0
    y1 = 212
    x2 = 300
    y2 = 0
    s1 = 0
    sg.plot_random_section_2_points(topo2[2:491,50:-50,:end_time], strat_ts, dx, x1, x2, y1, y2, s1, ve, 
        bottom = bottom, xoffset=0, yoffset=0, sea_level=sea_level_rs, subsid=subsid2[2:491,50:-50,:end_time], 
        linewidth=1, line_freq=2, color_mode='bathymetry', plot_type='3D', plot_basement=True)

    x1 = 0
    y1 = 212
    x2 = 0
    y2 = 268
    s1 = 0
    sg.plot_random_section_2_points(topo2[2:491,50:-50,:end_time], strat_ts, dx, x1, x2, y1, y2, s1, ve, 
        bottom = bottom, xoffset=0, yoffset=0, sea_level=sea_level_rs, subsid=subsid2[2:491,50:-50,:end_time], 
        linewidth=1, line_freq=2, color_mode='bathymetry', plot_type='3D', plot_basement=True)

    x1 = 300
    y1 = 0
    x2 = 899
    y2 = 0
    s1 = 0
    sg.plot_random_section_2_points(topo2[2:491,50:-50,:end_time], strat_ts, dx, x1, x2, y1, y2, s1, ve, 
        bottom = bottom, xoffset=0, yoffset=0, sea_level=sea_level_rs, subsid=subsid2[2:491,50:-50,:end_time], 
        linewidth=1, line_freq=2, color_mode='bathymetry', plot_type='3D', plot_basement=True)

    x1 = 300
    y1 = 488
    x2 = 899
    y2 = 488
    s1 = 0
    sg.plot_random_section_2_points(topo2[2:491,50:-50,:end_time], strat_ts, dx, x1, x2, y1, y2, s1, ve, 
        bottom = bottom, xoffset=0, yoffset=0, sea_level=sea_level_rs, subsid=subsid2[2:491,50:-50,:end_time], 
        linewidth=1, line_freq=2, color_mode='bathymetry', plot_type='3D', plot_basement=True)

    x1 = 899
    y1 = 0
    x2 = 899
    y2 = 488
    s1 = 0
    sg.plot_random_section_2_points(topo2[2:491,50:-50,:end_time], strat_ts, dx, x1, x2, y1, y2, s1, ve, 
        bottom = bottom, xoffset=0, yoffset=0, sea_level=sea_level_rs, subsid=subsid2[2:491,50:-50,:end_time], 
        linewidth=1, line_freq=2, color_mode='bathymetry', plot_type='3D', plot_basement=True)

    sg.plot_surf_w_texture(strat_ts, 1, dx, ve, im_pred[2:491,:].astype('float'), opacity=1.0, cmap='Blues', 
              mask = temp.astype('bool'), kmeans_colors=kmeans_colors)

    # plot bottom face of block:
    r,c,ts = np.shape(strat_ts)
    z = bottom*np.ones(np.shape(strat_ts[:,:,0]))
    z[temp==1] = np.nan
    X1 = scale*(np.linspace(0,c-1,c)*dx) # x goes with c and y with r
    Y1 = scale*(np.linspace(0,r-1,r)*dx)
    X1_grid , Y1_grid = np.meshgrid(X1, Y1)
    surf = mlab.mesh(X1_grid, Y1_grid, z*ve, scalars = z, mask = temp.astype('bool'), colormap='Blues', vmin=0, vmax=255)
    lut = surf.module_manager.scalar_lut_manager.lut.table.to_array()
    gray = np.expand_dims(colors.to_rgba('lightgray')[:3], axis=0)
    lut[:,:3] = 255*np.repeat(gray, 256, axis=0)
    surf.module_manager.scalar_lut_manager.lut.table = lut

    # plot water surface:
    sl = sea_level_rs[end_time-1]
    r,c,ts = np.shape(strat_ts)
    z = sl*np.ones(np.shape(strat_ts[:,:,0]))
    z[z < strat_ts[:, :, -1]] = np.nan
    X1 = scale*(np.linspace(0,c-1,c)*dx) # x goes with c and y with r
    Y1 = scale*(np.linspace(0,r-1,r)*dx)
    X1_grid , Y1_grid = np.meshgrid(X1, Y1)
    surf = mlab.mesh(X1_grid, Y1_grid, z*ve, scalars = np.zeros(np.shape(z)), mask = z < strat_ts[:, :, -1], 
                     colormap='Blues', vmin=0, vmax=255)
    lut = surf.module_manager.scalar_lut_manager.lut.table.to_array()
    blue = np.expand_dims(np.array([0.255, 0.412, 0.882]), axis=0)
    lut[:,:3] = 255*np.repeat(blue, 256, axis=0)
    lut[:,-1] = 255*0.3
    surf.module_manager.scalar_lut_manager.lut.table = lut

    mlab.view(azimuth=-60,
        elevation=66,
        distance=14000,
        focalpoint=np.array([ 5081,  2058, -2721]))

#     fname = '/Users/zoltan/Dropbox/Chronostratigraphy/XES02_3D_bathymetry_4_'+'%03d.png'%(end_time)
#     mlab.savefig(fname, magnification=2)

100%|██████████| 309/309 [00:15<00:00, 19.53it/s]


### Create 3D block diagram / exploded view with dip- and strike oriented sections

In [ ]:
end_time = 311
dx = 10.0

ind = np.argmin(np.abs(photo_times-time[end_time-1]))
fname = fnames[ind]
im = plt.imread(fname)
model, kmeans_colors = sg.pick_colors(im, 256)
im_pred = model.predict(im.reshape(-1, 3))
im_pred = im_pred.reshape(im.shape[0], im.shape[1])
im_pred = im_pred[::-1,:]

# account for subsidence:
topo_s = topo2[2:491,50:-50,:end_time].copy() 
for i in range(0, topo_s.shape[2]):
    topo_s[:,:,i] = topo_s[:,:,i]+(subsid2[2:491,50:-50,:end_time][:,:,-1]-subsid2[2:491,50:-50,:end_time][:,:,i])

bottom = -1300

# mlab.clf()
mlab.figure()

sg.create_exploded_view(topo2[2:491,50:-50,:end_time], sg.topostrat(topo_s), nx=2, ny=2, gap=200, dx=dx, ve=3, 
    color_mode='bathymetry', linewidth=0.5, bottom=bottom, opacity=1.0, x0=0, y0=0, 
    subsid=subsid2[2:491,50:-50,:end_time], texture=im_pred[2:491,:], sea_level=sea_level_rs[:end_time], 
    scale=1, plot_sides=True, plot_water=True, plot_surf=True, 
    surf_cmap='Blues', kmeans_colors=kmeans_colors, line_freq=1)

mlab.view(azimuth=-56,
    elevation=64,
    distance=16000,
    focalpoint=np.array([ 4574,  2646, -2847]))

### Create fence diagram

In [26]:
mlab.figure()

bottom = -1300
scale = 1
ve = 5

sg.create_fence_diagram(topo2[2:491,50:-50,:end_time], sg.topostrat(topo_s[2:491,50:-50,:end_time]), nx=4, ny=1, dx=dx, ve=ve, 
        color_mode='bathymetry', linewidth=0.5, bottom=bottom, opacity=0.7, subsid=subsid2[2:491,50:-50,:end_time],
        sea_level=sea_level_rs[:end_time], scale=scale, plot_sides=True, plot_water=True)

# mlab.view(azimuth=-50,
#     elevation=64,
#     distance=14175,
#     focalpoint=np.array([ 4078,  3515, -2918]))

100%|██████████| 3/3 [00:40<00:00, 13.52s/it]


(array([  0, 180, 360, 540, 720, 899]), array([  0, 244, 488]))